In [1]:
# Library ----------------------------------------------------------------
library(dplyr)
library(tidyr)

# 1. Load Data and Clustering Results ---------------------------
kmeans_result <- readRDS("../../../data/qc2022/preparation_donnees/03_pilot1_kmeans.rds")
df_pilot1_2022 <- readRDS("../../../data/qc2022/preparation_donnees/03_pilot1_2022.rds")

# write.csv(clusters_means, file = "../../../data/qc2022/preparation_donnees/03_pilot1_clusters_desc.csv", row.names = FALSE)